In [ ]:
import serial # You need pyserial for this
import json
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Make animation work in notebook
%matplotlib qt

## Manual setup

In [ ]:
port = '/dev/ttyACM1'
ser = serial.Serial(port, 115200, timeout=1)
time.sleep(2)

In [ ]:
data_to_send = "eargs -s 0"
ser.write(data_to_send.encode())

In [ ]:
received_data = ser.readline().decode().strip() 
print("Received: {}".format(received_data))

In [ ]:
# Detect port name
from serial.tools import list_ports
ports = list_ports.comports()
for port in ports:
    print(port.device)

## Real-time plotting

In [ ]:
# Serial connection setup
from serial.tools import list_ports
ports = list_ports.comports()
SERIAL_PORT = ports[0].device # Auto-detect; only works for one port
BAUD_RATE = 115200
TIME_WINDOW = 1000  # Sliding window width, in ms

time_data = []
adc_data = []

# Initialize serial connection
try:
    ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
    print(f"Connected to {SERIAL_PORT}")
except serial.SerialException as e:
    print(f"Error: Could not open serial port {SERIAL_PORT}. {e}")
    raise SystemExit

# Set up plot
fig, ax = plt.subplots()
line, = ax.plot([], [], lw=2)
latest_text = ax.text(0.9, 0.9, '', transform=ax.transAxes, fontsize=12, color='blue', bbox=dict(facecolor='white'))
ax.set_xlim(-TIME_WINDOW, 0)
ax.set_ylim(2500, 4096)
ax.set_xlabel("Time (s)")
ax.set_ylabel("ADC Value")
ax.grid(True)

# Update function for the animation
def update(frame):
    global time_data, adc_data

    # Read data from the serial port
    try:
        if ser.in_waiting > 0:
            line_data = ser.readline().decode('utf-8', errors='ignore').strip()
            parsed = json.loads(line_data)
            current_time, adc_value = parsed["time"], parsed["ADC"]

            time_data.append(current_time)
            adc_data.append(adc_value)

            # Keep only the last TIME_WINDOW ms of data
            time_data = [t for t in time_data if t >= current_time - TIME_WINDOW]
            adc_data = adc_data[-len(time_data):]
    except json.JSONDecodeError:
        print(f"Invalid JSON; read: {line_data}")
        pass  # Ignore invalid JSON lines
    except serial.SerialException as e:
        print(f"Serial exception occurred: {e}")
        ser.close()
        raise SystemExit

    # Update the plot
    if time_data and adc_data:
        relative_time = [t - time_data[-1] for t in time_data]  # Shift time to end at 0
        line.set_data(relative_time, adc_data)
        latest_text.set_text(f"Latest: {adc_data[-1]}")

    return line, latest_text

# Start the animation
ani = FuncAnimation(fig, update, interval=50)

# Show the plot
plt.show()

In [ ]:
ser.close()
print("Serial connection closed.") # Manual closing is required


In [ ]:
time_data, adc_data